In [35]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [36]:
dataset = pd.read_csv('Churn_Modelling.csv')

In [37]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
y = dataset.iloc[:,-1].values
X = dataset.iloc[:,3:-1].values

In [39]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

### LabelEcoding "Gender"

In [40]:
X[:,2] = le.fit_transform(X[:,2])

In [41]:
X[:,2]

array([0, 0, 0, ..., 0, 1, 0], dtype=object)

In [42]:
X[1]

array([608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58], dtype=object)

### OneHotEncoding

In [43]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [44]:
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [45]:
X[1]

array([0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
      dtype=object)

### Split to training and testing set

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,y, random_state=42, test_size=0.2)

### Feature Scaling

In [47]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [48]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [49]:
X_train

array([[ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
         0.97481699,  1.36766974],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699,  1.6612541 ],
       [-0.99850112, -0.57946723,  1.73494238, ...,  0.64920267,
        -1.02583358, -0.25280688],
       ...,
       [ 1.00150113, -0.57946723, -0.57638802, ..., -1.54035103,
        -1.02583358, -0.1427649 ],
       [ 1.00150113, -0.57946723, -0.57638802, ...,  0.64920267,
        -1.02583358, -0.05082558],
       [-0.99850112,  1.72572313, -0.57638802, ...,  0.64920267,
         0.97481699, -0.81456811]])

### ANN

In [50]:
NN = tf.keras.models.Sequential()

In [51]:
NN.add(tf.keras.layers.Dense(units=6, activation = 'relu'))
NN.add(tf.keras.layers.Dense(units=6, activation = 'relu'))
NN.add(tf.keras.layers.Dense(units=1, activation = 'sigmoid'))

### Training

In [52]:
NN.compile(optimizer= 'adam', loss= 'binary_crossentropy', metrics = ['accuracy'])

In [53]:
NN.fit(X_train, Y_train, batch_size = 32, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 154us/sample - loss: 0.6773 - acc: 0.6273
Epoch 2/100
8000/8000 [==============================] - 1s 124us/sample - loss: 0.4826 - acc: 0.7944
Epoch 3/100
8000/8000 [==============================] - 1s 126us/sample - loss: 0.4463 - acc: 0.8008
Epoch 4/100
8000/8000 [==============================] - 1s 128us/sample - loss: 0.4307 - acc: 0.8108
Epoch 5/100
8000/8000 [==============================] - 1s 134us/sample - loss: 0.4217 - acc: 0.8201
Epoch 6/100
8000/8000 [==============================] - 1s 127us/sample - loss: 0.4138 - acc: 0.8264
Epoch 7/100
8000/8000 [==============================] - 1s 135us/sample - loss: 0.4068 - acc: 0.8281
Epoch 8/100
8000/8000 [==============================] - 1s 126us/sample - loss: 0.4014 - acc: 0.8288
Epoch 9/100
8000/8000 [==============================] - 1s 135us/sample - loss: 0.3965 - acc: 0.8276
Epoch 10/100
8000/8000 [==============================] - 1s 128us/sample - loss: 

### Prediction

In [54]:
y_pred = NN.predict(X_test)>0.5
Results = np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test),1)),1)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test, y_pred)

[[1534   73]
 [ 205  188]]


0.861

In [55]:
Results

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [1, 1],
       [0, 1],
       [0, 1]])

In [56]:
NN.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))>0.5

array([[False]])